In [1]:
# Problem Statement: "The Automated Investment Risk Analyst"
# The Business Context: You are a Quantitative Researcher at a hedge fund. Your job is to monitor portfolio companies for risks. A sudden drop in revenue is a "lagging indicator"—by the time you see it, it's often too late. You need to connect these financial dips to operational events immediately.
# The Data Challenge: You have two disconnected datasets for a company called "Solaris AI":
# The Hard Numbers (financials.csv): A structured dataset containing quarterly Revenue, OpEx, and Churn Rates.
# Constraint: LLMs are bad at math. You cannot just ask "What is the average?" and trust the answer. You need an engine that can execute code.
# The Soft Signals (meeting_notes.txt): Unstructured text logs from the CEO's board meetings.
# Constraint: This file is too large to paste into a prompt every time. You need a retrieval system to find only the relevant paragraphs.
# The Goal: Build a unified LangChain Agent that acts as a Controller. When a user asks a complex question, the Agent must intelligently route the sub-tasks to the correct "Brain":
# Logic/Math: A Pandas Agent that writes Python code to query the CSV.
# Context/Search: A Vector Store Retriever (FAISS) that performs semantic search on the text.
# The Question:
# "Using the Financial Data, identify which Quarter had the highest Churn Rate. Then, use the Meeting Notes to explain the specific operational incident that caused this spike."


In [1]:
pip install langchain==0.1.7 langchain-community==0.0.20 langchain-experimental==0.0.51 langchain-core==0.1.23 sentence-transformers faiss-cpu pandas langchain-google-genai google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_experimental.agents import create_csv_agent
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents.agent_types import AgentType
from langchain.tools import Tool
from langchain.agents import initialize_agent

# 1) Load meeting notes
with open("solaris_meeting_notes.txt", "r", encoding="utf-8") as f:
    notes_text = f.read()

# 2) Embeddings (HF)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3) Semantic chunking
splitter = SemanticChunker(embeddings)
print(notes_text)
chunks = splitter.split_text(notes_text)

# 4) Store in FAISS
vs = FAISS.from_texts(chunks, embeddings)
vs.save_local("meeting_notes_index")

print("✅ Saved semantic chunks to meeting_notes_index")

/Users/devenderswami/lc-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CONFIDENTIAL BOARD MEETING MINUTES - SOLARIS AI
ARCHIVE: 2021-2025
--------------------------------------------------


DATE: 2021-01-01
ATTENDEES: CEO, CTO, CFO, Board Members
SUMMARY: The board reviewed the performance for Q1 2021.
Operational efficiency remains a key priority. The engineering team is deploying new microservices. Sales team is pushing for Q-end quotas. Standard operations continuing. No major incidents to report. Morale is stable.
------------------------------
DATE: 2021-02-01
ATTENDEES: CEO, CTO, CFO, Board Members
SUMMARY: The board reviewed the performance for Q1 2021.
Operational efficiency remains a key priority. The engineering team is deploying new microservices. Sales team is pushing for Q-end quotas. Standard operations continuing. No major incidents to report. Morale is stable.
------------------------------
DATE: 2021-03-01
ATTENDEES: CEO, CTO, CFO, Board Members
SUMMARY: The board reviewed the performance for Q1 2021.
Operational efficiency remains a key

In [49]:
df = pd.read_csv("solaris_financials.csv")
os.environ["GOOGLE_API_KEY"] = ""
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    temperature=0.2
)
# df["Quarter"] = df["Quarter"].astype(str).str.upper().str.strip()
# print(df)

csv_agent = create_csv_agent(
    llm,
    'solaris_financials.csv',
    verbose=True,
    prefix="give the output as text",
    agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [50]:
def financial_tool_func(query: str):
    return csv_agent.run(query)
    

financial_tool = Tool(
    name="financial_analyst",
    func=financial_tool_func,
    description="Use for numeric questions: revenue, churn, profit, averages, trends, comparisons."
)

In [51]:
def meeting_notes_func(query: str):
    retriever = vs.as_retriever(search_kwargs={"k": 4})
    docs = retriever.get_relevant_documents(query)
    return "\n".join([d.page_content for d in docs])

notes_tool = Tool(
    name="meeting_notes_search",
    func=meeting_notes_func,
    description="Use for operational context, incidents, reasons, explanations from internal meeting notes."
)

In [52]:
controller = initialize_agent(
    tools=[financial_tool, notes_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [53]:
print(controller("Which quarter had the highest churn and why?"))
#controller.run("Which quarter had the highest churn and why?")



> Entering new AgentExecutor chain...
Action: financial_analyst
Action Input: {"question": "Which quarter had the highest churn?"}

> Entering new AgentExecutor chain...
Action: python_repl_ast
Action Input:
import pandas as pd
from io import StringIO

data = """
| | Date | Year | Quarter | Revenue ($M) | OpEx ($M) | Net_Income ($M) | Churn_Rate (%) | Active_Users |
|---:|:-----------|-------:|:----------|---------------:|------------:|------------------:|-----------------:|---------------:|
| 0 | 2021-01-01 | 2021 | Q1 | 11.82 | 7.78 | 4.05 | 2.87 | 10100 |
| 1 | 2021-02-01 | 2021 | Q1 | 11.69 | 7.6 | 4.08 | 2.81 | 10200 |
| 2 | 2021-03-01 | 2021 | Q1 | 11.45 | 7.11 | 4.34 | 2.32 | 10300 |
| 3 | 2021-04-01 | 2021 | Q2 | 13.89 | 8.86 | 5.03 | 2.75 | 10400 |
| 4 | 2021-05-01 | 2021 | Q2 | 15.21 | 11.94 | 3.28 | 2.97 | 10500 |
"""

df = pd.read_csv(StringIO(data), sep='|', index_col=1).dropna(axis=1, how='all').iloc[1:]
df.columns = [col.strip() for col in df.columns]
df = df.apply(lam

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The quarter with the highest churn was Q2. This was due to a 'Blackout' incident, which resulted in a 10% churn spike and the loss of 3 Enterprise clients (Acme Corp, Globex, Soylent).`

In [37]:
# def classify_query(query: str) -> str:
#     q = query.lower()

#     has_number = any(w in q for w in [
#         "highest", "lowest", "average", "trend", "drop", "increase", "decrease",
#         "revenue", "churn", "profit", "margin", "net income", "opex"
#     ])

#     has_reason = any(w in q for w in [
#         "why", "reason", "because", "due to", "cause", "explain"
#     ])

#     if has_number and has_reason:
#         return "HYBRID"
#     if has_number:
#         return "FINANCIAL"
#     return "CONTEXT"

In [38]:
# def safe_llm_call(prompt: str):
#     try:
#         result = llm.invoke(prompt)
#         if result is None or str(result).strip() == "":
#             return "⚠️ LLM returned empty response.\n\n" + prompt
#         return llm.invoke(prompt)
#     except Exception as e:
#         return f"⚠️ LLM failed: {e}"

In [39]:
# # controller = initialize_agent(
# #     tools=[financial_tool, notes_tool],
# #     llm=llm,
# #     agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
# #     verbose=True
# # )
# def run_financial(query):
#     return financial_tool.run(query)


# def run_context(query):
#     return notes_tool.run(query)


# def run_hybrid(query):
#     numeric = financial_tool.run(query)

#     import re
#     m = re.search(r"Q[1-4]", numeric)
#     quarter = m.group() if m else None

#     context_query = f"What operational events were discussed in {quarter}?" if quarter else query
#     context = notes_tool.run(context_query)

#     prompt = f"""
# Financial Analysis:
# {numeric}

# Meeting Notes:
# {context}

# Explain clearly what caused this.
# """
#     return safe_llm_call(prompt)

In [40]:
# def controller(query: str):
#     intent = classify_query(query)

#     if intent == "FINANCIAL":
#         return run_financial(query)

#     if intent == "CONTEXT":
#         return run_context(query)

#     if intent == "HYBRID":
#         return run_hybrid(query)

#     return "Could not classify query."

In [44]:
print(controller("Which quarter had the highest churn rate?"))



> Entering new AgentExecutor chain...
Action: financial_analyst
Action Input: churn rate per quarter

> Entering new AgentExecutor chain...
Action: python_repl_ast
Action Input: print(df.groupby(['Year', 'Quarter'])['Churn_Rate (%)'].mean())Year  Quarter
2021  Q1         2.666667
      Q2         2.650000
      Q3         2.570000
      Q4         2.653333
2022  Q1         2.573333
      Q2         2.693333
      Q3         2.356667
      Q4         2.273333
2023  Q1         1.953333
      Q2         2.033333
      Q3         2.023333
      Q4         1.960000
2024  Q1         3.253333
      Q2         2.413333
      Q3         8.910000
      Q4         2.796667
2025  Q1         1.203333
      Q2         1.586667
      Q3         1.733333
      Q4         1.483333
Name: Churn_Rate (%), dtype: float64
I now know the final answer
Final Answer: The churn rate per quarter is as follows:
Year  Quarter
2021  Q1         2.67%
      Q2         2.65%
      Q3         2.57%
      Q4         2.

In [16]:
print(controller("What incident happened in Q3?"))

No major incidents to report. Morale is stable. ------------------------------
DATE: 2021-09-01
ATTENDEES: CEO, CTO, CFO, Board Members
SUMMARY: The board reviewed the performance for Q3 2021. Operational efficiency remains a key priority. The engineering team is deploying new microservices. Sales team is pushing for Q-end quotas. Standard operations continuing.
No major incidents to report. Morale is stable. ------------------------------
DATE: 2021-08-01
ATTENDEES: CEO, CTO, CFO, Board Members
SUMMARY: The board reviewed the performance for Q3 2021. Operational efficiency remains a key priority. The engineering team is deploying new microservices. Sales team is pushing for Q-end quotas. Standard operations continuing.
No major incidents to report. Morale is stable. ------------------------------
DATE: 2024-06-01
ATTENDEES: CEO, CTO, CFO, Board Members
SUMMARY: The board reviewed the performance for Q2 2024. Operational efficiency remains a key priority. The engineering team is deploy

In [41]:
print(controller("Which quarter had the highest churn and why?"))



> Entering new AgentExecutor chain...
Action: python_repl_ast
Action Input: print(df.groupby('Quarter')['Churn_Rate (%)'].mean().sort_values(ascending=False))Quarter
Q3    3.518667
Q1    2.330000
Q2    2.275333
Q4    2.233333
Name: Churn_Rate (%), dtype: float64


IndexError: list index out of range